# Insight Agent - Intelligent Business Analysis

This notebook implements a sophisticated AI agent that analyzes text and extracts structured business insights, issues, and recommendations.

## Capabilities

- **Analyze** customer escalations, incidents, and business documents
- **Extract** structured insights with severity classification
- **Perform** root cause analysis
- **Generate** actionable recommendations with priorities
- **Provide** urgency scoring and escalation flags

## Architecture

The agent uses:
- **LangChain** for orchestration
- **OpenAI GPT-4** for analysis
- **Pydantic** for structured output validation
- **JSON Schema** for consistent data formatting

## 1. Setup and Installation

First, we'll install the required packages and import necessary libraries.

In [1]:
# Install required packages (uncomment if needed)
!pip install langchain langchain-openai langchain-core pydantic openai python-dotenv

  Using cached langchain-1.0.7-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_openai-1.0.3-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_core-1.0.5-py3-none-any.whl.metadata (3.6 kB)
  Using cached pydantic-2.12.4-py3-none-any.whl.metadata (89 kB)
  Using cached openai-2.8.1-py3-none-any.whl.metadata (29 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.4 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached langgraph_sdk-0.2.9-py3-none-any.whl.metadata (1.5 kB)
  Using cached htt

In [2]:
# Import required libraries
import json
import os
from datetime import datetime
from typing import List, Optional, Dict, Any

# Pydantic for data validation
from pydantic import BaseModel, Field

# LangChain components
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_openai import ChatOpenAI

print("✅ All imports successful")

✅ All imports successful


## 2. Configure OpenAI API Key

Set your OpenAI API key to enable the agent to communicate with GPT-4.

**Options:**
1. Set environment variable directly
2. Load from `.env` file
3. Set via system environment

In [3]:
# Option 1: Set directly (not recommended for production)
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY_HERE'

# Option 2: Load from .env file (recommended)
from dotenv import load_dotenv
load_dotenv()

# Verify API key is set
if not os.environ.get('OPENAI_API_KEY'):
    print("⚠️  Warning: OPENAI_API_KEY not set. Please set it before running the analysis.")
else:
    print("✅ OpenAI API Key found")

✅ OpenAI API Key found


## 3. Define Data Models

We use Pydantic models to ensure the AI output is structured and validated. This provides:
- Type safety
- Automatic validation
- Clear schema definition
- Easy JSON serialization

In [4]:
class Issue(BaseModel):
    """Represents a single identified issue"""
    description: str = Field(description="Clear, specific description of the issue")
    severity: str = Field(description="Severity: Low, Medium, High, or Critical")
    category: str = Field(description="Category: Technical, Business, Process, People, or Other")
    impact: str = Field(description="Describe the impact of this issue")


class Insight(BaseModel):
    """Represents a key insight or observation"""
    insight: str = Field(description="The key insight or pattern identified")
    supporting_evidence: str = Field(description="Evidence from the text supporting this insight")
    confidence: str = Field(description="Confidence level: Low, Medium, or High")
    implications: str = Field(description="What this insight means for the business")


class RootCause(BaseModel):
    """Root cause analysis results"""
    primary_cause: str = Field(description="The identified primary root cause")
    reasoning: str = Field(description="Detailed explanation of why this is the root cause")
    contributing_factors: List[str] = Field(description="Additional factors that contributed")
    evidence: List[str] = Field(description="Evidence supporting this root cause")


class Action(BaseModel):
    """Represents a single actionable recommendation"""
    action: str = Field(description="Specific, actionable step")
    owner: str = Field(description="Suggested owner or department responsible")
    priority: str = Field(description="Priority: Low, Medium, High, or Urgent")
    timeline: str = Field(description="Suggested timeline for completion")
    expected_outcome: str = Field(description="What success looks like")
    dependencies: List[str] = Field(default_factory=list, description="Prerequisites or dependencies")


class InsightAgentOutput(BaseModel):
    """Complete output from the Insight Agent"""
    executive_summary: str = Field(description="2-3 sentence executive summary")
    key_issues: List[Issue] = Field(description="All identified issues with severity")
    insights: List[Insight] = Field(description="Key insights and patterns")
    root_cause_analysis: Optional[RootCause] = Field(
        default=None, 
        description="Root cause analysis if this is an incident or problem"
    )
    recommended_actions: List[Action] = Field(description="Prioritized action items")
    urgency_score: int = Field(
        ge=1, le=10,
        description="Overall urgency from 1-10 based on impact and severity"
    )
    requires_escalation: bool = Field(description="Whether this requires management escalation")
    estimated_impact: str = Field(description="Estimated business impact: Low, Medium, High, or Critical")
    metadata: Dict[str, Any] = Field(default_factory=dict, description="Additional metadata")

print("✅ Pydantic models defined successfully")

✅ Pydantic models defined successfully


## 4. Build the Insight Agent

The `SimpleInsightAgent` class encapsulates the AI analysis logic. It:

1. **Creates a structured prompt** with detailed analysis requirements
2. **Chains together** LangChain components (prompt → LLM → parser)
3. **Validates output** using Pydantic models
4. **Adds metadata** for tracking and auditing

### Key Design Decision

We use `StrOutputParser()` before `PydanticOutputParser()` to extract the text content from the LLM response before parsing it as JSON. This prevents type errors.

In [5]:
class SimpleInsightAgent:
    """
    AI-powered insight agent that analyzes text and extracts structured insights.
    
    Features:
    - Executive summary generation
    - Issue identification with severity classification
    - Pattern and insight detection
    - Root cause analysis
    - Actionable recommendation generation
    - Urgency scoring and escalation flags
    """
    
    def __init__(self, llm):
        """
        Initialize the agent with an LLM.
        
        Args:
            llm: LangChain LLM instance (e.g., ChatOpenAI)
        """
        self.llm = llm
        self.parser = PydanticOutputParser(pydantic_object=InsightAgentOutput)
        
        # Create prompt template with detailed analysis instructions
        self.prompt = ChatPromptTemplate.from_messages([
            ("human", """
            You are an expert business analyst specializing in operational analysis and risk assessment.

            Your task is to perform a comprehensive analysis of the following text and extract actionable insights.

            TEXT TO ANALYZE:
            {text}

            ANALYSIS REQUIREMENTS:

            1. EXECUTIVE SUMMARY (2-3 sentences)
               - What is this about?
               - What is the main concern?
               - What is at stake?

            2. KEY ISSUES
               - Identify ALL distinct issues mentioned or implied
               - Classify severity: Low, Medium, High, or Critical
               - Categorize: Technical, Business, Process, People, or Other
               - Describe impact for each issue

            3. INSIGHTS & PATTERNS
               - What patterns do you observe?
               - What are the underlying themes?
               - What implications does this have?
               - Provide supporting evidence from the text
               - Indicate confidence level for each insight

            4. ROOT CAUSE ANALYSIS (if applicable)
               - If this describes a problem or incident, identify the PRIMARY root cause
               - Explain your reasoning with evidence
               - List contributing factors
               - If root cause is unclear, acknowledge this

            5. RECOMMENDED ACTIONS
               - Provide specific, actionable steps
               - Assign suggested owners (departments or roles)
               - Set priorities: Low, Medium, High, or Urgent
               - Suggest timelines
               - Describe expected outcomes
               - Note any dependencies

            6. URGENCY & ESCALATION
               - Assign urgency score (1-10) based on:
                 * Severity of issues
                 * Business impact
                 * Time sensitivity
               - Determine if escalation to management is needed
               - Estimate overall business impact: Low, Medium, High, or Critical

            IMPORTANT GUIDELINES:
            - Base ALL analysis on information in the text
            - Be specific, not generic
            - Prioritize actionability over description
            - If information is missing, acknowledge it
            - Use business language, not technical jargon

            {format_instructions}

            Output ONLY valid JSON. No additional text.
            """)
        ])
        
        # Build the processing chain
        # StrOutputParser extracts text content from LLM message
        # PydanticOutputParser then parses JSON into structured model
        self.chain = (
            self.prompt 
            | self.llm 
            | StrOutputParser()  # Extract text content
            | self.parser         # Parse JSON into Pydantic model
        )
    
    def analyze(self, text: str) -> InsightAgentOutput:
        """
        Analyze text and return structured insights.
        
        Args:
            text: Text to analyze (customer feedback, incident report, etc.)
            
        Returns:
            InsightAgentOutput: Structured analysis results
        """
        result = self.chain.invoke({
            "text": text,
            "format_instructions": self.parser.get_format_instructions()
        })
        
        # Add metadata for tracking
        result.metadata["analysis_date"] = datetime.now().isoformat()
        result.metadata["source"] = "simple_agent"
        result.metadata["text_length"] = len(text)
        
        return result

print("✅ SimpleInsightAgent class defined")

✅ SimpleInsightAgent class defined


## 5. Create Display Helper Function

This function provides a formatted, readable output of the analysis results with color-coded emojis for easy scanning.

In [6]:
def display_insight_analysis(result: InsightAgentOutput, title: str = "INSIGHT ANALYSIS"):
    """
    Pretty print the analysis results with formatting and emojis.
    
    Args:
        result: InsightAgentOutput from the agent
        title: Title for the analysis display
    """
    print("=" * 80)
    print(f" {title}")
    print("=" * 80)
    
    # Executive Summary
    print("\n📊 EXECUTIVE SUMMARY")
    print("-" * 80)
    print(result.executive_summary)
    
    # Urgency & Impact
    print(f"\n⚠️  URGENCY SCORE: {result.urgency_score}/10")
    print(f"📈 ESTIMATED IMPACT: {result.estimated_impact}")
    print(f"🚨 REQUIRES ESCALATION: {'YES' if result.requires_escalation else 'NO'}")
    
    # Key Issues
    print(f"\n🔍 KEY ISSUES ({len(result.key_issues)} identified)")
    print("-" * 80)
    for i, issue in enumerate(result.key_issues, 1):
        severity_emoji = {
            "Critical": "🔴",
            "High": "🟠",
            "Medium": "🟡",
            "Low": "🟢"
        }.get(issue.severity, "⚪")
        
        print(f"\n{i}. {severity_emoji} {issue.severity.upper()} - {issue.category}")
        print(f"   Description: {issue.description}")
        print(f"   Impact: {issue.impact}")
    
    # Insights
    print(f"\n💡 KEY INSIGHTS ({len(result.insights)} identified)")
    print("-" * 80)
    for i, insight in enumerate(result.insights, 1):
        confidence_emoji = {
            "High": "✅",
            "Medium": "⚠️",
            "Low": "❓"
        }.get(insight.confidence, "⚪")
        
        print(f"\n{i}. {confidence_emoji} {insight.insight}")
        print(f"   Confidence: {insight.confidence}")
        print(f"   Evidence: {insight.supporting_evidence}")
        print(f"   Implications: {insight.implications}")
    
    # Root Cause Analysis
    if result.root_cause_analysis:
        print("\n🎯 ROOT CAUSE ANALYSIS")
        print("-" * 80)
        print(f"Primary Cause: {result.root_cause_analysis.primary_cause}")
        print(f"\nReasoning: {result.root_cause_analysis.reasoning}")
        
        if result.root_cause_analysis.contributing_factors:
            print("\nContributing Factors:")
            for factor in result.root_cause_analysis.contributing_factors:
                print(f"  • {factor}")
        
        if result.root_cause_analysis.evidence:
            print("\nSupporting Evidence:")
            for evidence in result.root_cause_analysis.evidence:
                print(f"  • {evidence}")
    
    # Recommended Actions
    print(f"\n✅ RECOMMENDED ACTIONS ({len(result.recommended_actions)} actions)")
    print("-" * 80)
    for i, action in enumerate(result.recommended_actions, 1):
        priority_emoji = {
            "Urgent": "🔴",
            "High": "🟠",
            "Medium": "🟡",
            "Low": "🟢"
        }.get(action.priority, "⚪")
        
        print(f"\n{i}. {priority_emoji} [{action.priority}] {action.action}")
        print(f"   Owner: {action.owner}")
        print(f"   Timeline: {action.timeline}")
        print(f"   Expected Outcome: {action.expected_outcome}")
        
        if action.dependencies:
            print(f"   Dependencies: {', '.join(action.dependencies)}")
    
    print("\n" + "=" * 80)

print("✅ Display helper function defined")

✅ Display helper function defined


## 6. Initialize the Agent

Create an instance of the agent with GPT-4 configured for optimal analysis performance.

**Configuration:**
- **Model:** GPT-4o (balanced performance and cost)
- **Temperature:** 0.3 (lower = more consistent)
- **Response Format:** JSON mode (ensures valid JSON output)

In [7]:
# Initialize LLM with appropriate settings
llm = ChatOpenAI(
    model="gpt-4o",  # Options: "gpt-4o", "gpt-4-turbo", "gpt-3.5-turbo"
    temperature=0.3,  # Lower temperature for more consistent analysis
    model_kwargs={"response_format": {"type": "json_object"}}  # Ensure JSON output
)

# Create the agent
simple_agent = SimpleInsightAgent(llm)

print("✅ Insight Agent initialized and ready")

✅ Insight Agent initialized and ready


## 7. Test Case 1: Customer Escalation Analysis

This example analyzes a critical customer escalation from a high-value enterprise account experiencing service issues.

**Scenario:** Fortune 500 customer threatening to cancel a $2.4M contract due to:
- Multiple service outages
- Poor support response times
- Undelivered features
- Financial and reputational damage

In [8]:
customer_escalation = """
URGENT ESCALATION - Enterprise Account
Customer: TechCorp Global (Account #A-7845)
Date: January 20, 2024
Reported by: Jennifer Martinez, CTO

We are writing to formally escalate serious concerns about your platform's reliability and support.
Over the past 30 days, we have experienced FOUR complete system outages affecting our entire
operations. Each incident has resulted in:

1. Complete loss of platform access (longest: 6 hours)
2. Data sync failures causing reporting inconsistencies
3. Lost productivity estimated at $75,000 per incident
4. Damage to our reputation with our own customers

What's particularly concerning:
- We receive no proactive communication about outages
- Support response time averages 48-72 hours
- Root cause analyses are vague and unhelpful
- Promised features from our sales cycle remain undelivered

TechCorp represents 18% of our annual revenue ($2.4M contract) and serves as a reference account
for our industry. We are a Fortune 500 company with strict SLA requirements.

We require within 24 hours:
1. Detailed root cause analysis for ALL four outages
2. Concrete prevention plan with measurable milestones
3. Service credits totaling $300,000
4. Executive-level meeting with your VP of Customer Success
5. Assigned dedicated support engineer

Without immediate action, we will:
- Invoke the termination clause in our contract
- Explore alternative vendors (RFP already in progress)
- Share our experience publicly in industry forums

Our renewal is in 90 days. We need to see dramatic improvement or we will not renew.

Jennifer Martinez
Chief Technology Officer
TechCorp Global
"""

print("Analyzing customer escalation...\n")
result1 = simple_agent.analyze(customer_escalation)
display_insight_analysis(result1, "CUSTOMER ESCALATION ANALYSIS")

Analyzing customer escalation...

 CUSTOMER ESCALATION ANALYSIS

📊 EXECUTIVE SUMMARY
--------------------------------------------------------------------------------
The text is an urgent escalation from TechCorp Global regarding severe reliability and support issues with the platform. The main concern is the repeated system outages and inadequate support response, which threaten a significant revenue source and reference account. At stake is the potential loss of a $2.4M contract, damage to reputation, and the risk of public exposure of the issues.

⚠️  URGENCY SCORE: 10/10
📈 ESTIMATED IMPACT: Critical
🚨 REQUIRES ESCALATION: YES

🔍 KEY ISSUES (5 identified)
--------------------------------------------------------------------------------

1. 🔴 CRITICAL - Technical
   Description: Four complete system outages in the last 30 days.
   Impact: Complete loss of platform access, data sync failures, and significant financial and reputational damage.

2. 🟠 HIGH - Process
   Description: No pro

### Save Analysis to JSON

Export the structured results for integration with other systems or databases.

In [9]:
# Convert to dictionary and save as JSON
result_dict = result1.model_dump()
result_json = json.dumps(result_dict, indent=2)

# Display preview
print("=" * 80)
print("JSON OUTPUT (for API/Database integration):")
print("=" * 80)
if len(result_json) > 1000:
    print(result_json[:1000] + "...\n[truncated]")
    print(f"\nFull JSON length: {len(result_json)} characters")
else:
    print(result_json)

# Save to file
output_file = "customer_escalation_analysis.json"
with open(output_file, "w") as f:
    json.dump(result_dict, f, indent=2)
print(f"\n✅ Full analysis saved to: {output_file}")

JSON OUTPUT (for API/Database integration):
{
  "executive_summary": "The text is an urgent escalation from TechCorp Global regarding severe reliability and support issues with the platform. The main concern is the repeated system outages and inadequate support response, which threaten a significant revenue source and reference account. At stake is the potential loss of a $2.4M contract, damage to reputation, and the risk of public exposure of the issues.",
  "key_issues": [
    {
      "description": "Four complete system outages in the last 30 days.",
      "severity": "Critical",
      "category": "Technical",
      "impact": "Complete loss of platform access, data sync failures, and significant financial and reputational damage."
    },
    {
      "description": "No proactive communication about outages.",
      "severity": "High",
      "category": "Process",
      "impact": "Increased customer frustration and lack of trust in the service provider."
    },
    {
      "descriptio

## 8. Test Case 2: Production Incident Report

This example analyzes a critical production outage affecting payment processing.

**Scenario:** 3.8-hour outage causing:
- $420,000 revenue loss
- 15,000+ support tickets
- Negative social media impact
- Database connection pool exhaustion

In [10]:
production_incident = """
INCIDENT REPORT - P1 Production Outage
Incident ID: INC-2024-0156
Date: January 18, 2024, 14:23 UTC
Duration: 3 hours 47 minutes
Affected Services: Payment Processing, User Authentication

IMPACT:
- 100% of payment transactions failed during outage window
- Estimated revenue loss: $420,000
- 15,000+ customer support tickets created
- Social media sentiment turned negative (-67% shift)

TIMELINE:
14:23 - Database connection pool exhausted, services start failing
14:45 - PagerDuty alerts trigger, on-call engineer engaged
15:12 - Issue identified as runaway query from new feature deployment
15:30 - Attempted rollback failed due to migration dependencies
16:45 - Manual database cleanup initiated
18:10 - Services fully restored

ROOT CAUSE:
New recommendation engine deployed at 13:00 contained unoptimized query that 
created N+1 problem. Load testing in staging missed this because test data 
volume was only 1% of production.

CONTRIBUTING FACTORS:
- Insufficient staging environment
- No query performance monitoring in place
- Rollback procedures not tested
- No gradual rollout strategy for new features
"""

print("Analyzing production incident...\n")
result2 = simple_agent.analyze(production_incident)
display_insight_analysis(result2, "PRODUCTION INCIDENT ANALYSIS")

Analyzing production incident...

 PRODUCTION INCIDENT ANALYSIS

📊 EXECUTIVE SUMMARY
--------------------------------------------------------------------------------
The incident report describes a critical production outage affecting payment processing and user authentication services, resulting in significant revenue loss and negative customer sentiment. The main concern is the unoptimized query from a new feature deployment that led to a database connection pool exhaustion. At stake is the company's financial performance and customer trust.

⚠️  URGENCY SCORE: 9/10
📈 ESTIMATED IMPACT: Critical
🚨 REQUIRES ESCALATION: YES

🔍 KEY ISSUES (6 identified)
--------------------------------------------------------------------------------

1. 🔴 CRITICAL - Technical
   Description: Database connection pool exhaustion due to unoptimized query.
   Impact: Caused 100% failure of payment transactions and significant revenue loss.

2. 🟠 HIGH - Technical
   Description: Runaway query from new feature

## 9. Test Case 3: Employee Feedback Analysis

This example analyzes quarterly employee survey results showing concerning trends.

**Scenario:** Engineering department survey reveals:
- Declining work-life balance scores
- Burnout from context switching
- Frequent scope changes disrupting sprints
- Recent departures citing unsustainable pace

In [ ]:
employee_feedback = """
QUARTERLY SURVEY - Engineering Department Results
Q4 2023 - 127 Responses (85% participation rate)

SATISFACTION SCORES (1-10 scale):
- Work-life balance: 4.2 (down from 7.1 in Q3)
- Management communication: 5.8 (down from 6.9)
- Career growth opportunities: 6.1 (stable)
- Technical challenges: 8.2 (up from 7.8)
- Team collaboration: 7.9 (stable)

RECURRING THEMES FROM OPEN RESPONSES:

"The constant context switching between projects is burning everyone out. 
I'm working on 4 different initiatives simultaneously." - 23 similar comments

"We keep getting urgent requests from sales that derail our sprint plans. 
There's no buffer for unplanned work." - 18 similar comments

"I love the technical problems we're solving, but the pace is unsustainable. 
Three teammates quit in the last month citing burnout." - 31 similar comments

"Leadership keeps saying 'work-life balance is important' but then expects 
us to deliver impossible timelines." - 15 similar comments

POSITIVE FEEDBACK:
"The new tech stack is great to work with" - 42 mentions
"Team members are talented and supportive" - 38 mentions
"Product vision is exciting" - 27 mentions
"""

print("Analyzing employee feedback...\n")
result3 = simple_agent.analyze(employee_feedback)
display_insight_analysis(result3, "EMPLOYEE FEEDBACK ANALYSIS")

## 10. Utility Functions

Helper functions to filter and extract specific types of information from analysis results.

In [ ]:
def get_critical_issues(result: InsightAgentOutput) -> List[Issue]:
    """Extract only critical severity issues"""
    return [issue for issue in result.key_issues if issue.severity == "Critical"]

def get_urgent_actions(result: InsightAgentOutput) -> List[Action]:
    """Extract only urgent priority actions"""
    return [action for action in result.recommended_actions if action.priority == "Urgent"]

def get_high_confidence_insights(result: InsightAgentOutput) -> List[Insight]:
    """Extract only high confidence insights"""
    return [insight for insight in result.insights if insight.confidence == "High"]

# Example usage
print("=" * 80)
print("FILTERED ANALYSIS - Customer Escalation")
print("=" * 80)

critical_issues = get_critical_issues(result1)
print(f"\n🔴 CRITICAL ISSUES ONLY ({len(critical_issues)}):")
for i, issue in enumerate(critical_issues, 1):
    print(f"{i}. {issue.description}")

urgent_actions = get_urgent_actions(result1)
print(f"\n🔴 URGENT ACTIONS ONLY ({len(urgent_actions)}):")
for i, action in enumerate(urgent_actions, 1):
    print(f"{i}. {action.action} (Owner: {action.owner})")

## 11. Batch Processing

Process multiple documents in a single run for efficiency.

In [ ]:
def batch_analyze(texts: List[str], descriptions: List[str]) -> List[InsightAgentOutput]:
    """
    Analyze multiple texts in batch.
    
    Args:
        texts: List of texts to analyze
        descriptions: List of descriptions for each text
        
    Returns:
        List of InsightAgentOutput results
    """
    results = []
    for i, (text, desc) in enumerate(zip(texts, descriptions), 1):
        print(f"\n{'='*80}")
        print(f"Processing {i}/{len(texts)}: {desc}")
        print(f"{'='*80}")
        result = simple_agent.analyze(text)
        results.append(result)
        print(f"✅ Completed - Urgency: {result.urgency_score}/10, Issues: {len(result.key_issues)}")
    
    return results

# Example: Uncomment to run batch processing
# batch_texts = [customer_escalation, production_incident, employee_feedback]
# batch_descriptions = ["Customer Escalation", "Production Incident", "Employee Feedback"]
# batch_results = batch_analyze(batch_texts, batch_descriptions)

print("✅ Batch processing function defined")

## 12. Export Functions

Export analysis results to different formats for reporting and integration.

In [ ]:
def export_to_csv(result: InsightAgentOutput, filename: str):
    """Export issues and actions to CSV format"""
    import csv
    
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        
        # Write issues
        writer.writerow(['Type', 'Severity/Priority', 'Category/Owner', 'Description/Action', 'Impact/Outcome'])
        writer.writerow([])
        writer.writerow(['ISSUES'])
        for issue in result.key_issues:
            writer.writerow(['Issue', issue.severity, issue.category, issue.description, issue.impact])
        
        writer.writerow([])
        writer.writerow(['ACTIONS'])
        for action in result.recommended_actions:
            writer.writerow(['Action', action.priority, action.owner, action.action, action.expected_outcome])
    
    print(f"✅ Exported to CSV: {filename}")


def export_to_markdown(result: InsightAgentOutput, filename: str):
    """Export analysis to Markdown format"""
    with open(filename, 'w') as f:
        f.write(f"# Insight Analysis Report\n\n")
        f.write(f"**Analysis Date:** {result.metadata.get('analysis_date', 'N/A')}\n\n")
        f.write(f"**Urgency Score:** {result.urgency_score}/10\n\n")
        f.write(f"**Estimated Impact:** {result.estimated_impact}\n\n")
        f.write(f"## Executive Summary\n\n{result.executive_summary}\n\n")
        
        f.write(f"## Key Issues ({len(result.key_issues)})\n\n")
        for i, issue in enumerate(result.key_issues, 1):
            f.write(f"### {i}. [{issue.severity}] {issue.category}\n")
            f.write(f"**Description:** {issue.description}\n\n")
            f.write(f"**Impact:** {issue.impact}\n\n")
        
        f.write(f"## Recommended Actions ({len(result.recommended_actions)})\n\n")
        for i, action in enumerate(result.recommended_actions, 1):
            f.write(f"### {i}. [{action.priority}] {action.action}\n")
            f.write(f"- **Owner:** {action.owner}\n")
            f.write(f"- **Timeline:** {action.timeline}\n")
            f.write(f"- **Expected Outcome:** {action.expected_outcome}\n\n")
    
    print(f"✅ Exported to Markdown: {filename}")

# Example exports
export_to_csv(result1, "customer_escalation_analysis.csv")
export_to_markdown(result1, "customer_escalation_analysis.md")

## 13. Summary Statistics

Generate aggregate statistics across multiple analyses for trending and reporting.

In [ ]:
def generate_summary_stats(results: List[InsightAgentOutput]) -> Dict[str, Any]:
    """Generate aggregate statistics across multiple analyses"""
    stats = {
        "total_analyses": len(results),
        "avg_urgency_score": sum(r.urgency_score for r in results) / len(results),
        "total_issues": sum(len(r.key_issues) for r in results),
        "total_actions": sum(len(r.recommended_actions) for r in results),
        "escalation_required_count": sum(1 for r in results if r.requires_escalation),
        "impact_distribution": {},
        "severity_distribution": {}
    }
    
    # Impact distribution
    for result in results:
        impact = result.estimated_impact
        stats["impact_distribution"][impact] = stats["impact_distribution"].get(impact, 0) + 1
    
    # Severity distribution
    for result in results:
        for issue in result.key_issues:
            severity = issue.severity
            stats["severity_distribution"][severity] = stats["severity_distribution"].get(severity, 0) + 1
    
    return stats

# Example with our test cases
all_results = [result1, result2, result3]
summary = generate_summary_stats(all_results)

print("=" * 80)
print("SUMMARY STATISTICS - All Analyses")
print("=" * 80)
print(json.dumps(summary, indent=2))

## 14. How To Use

### Quick Usage

```python
# Analyze any text
result = simple_agent.analyze(your_text)

# Display formatted output
display_insight_analysis(result)

# Export to different formats
export_to_json(result, "analysis.json")
export_to_csv(result, "analysis.csv")
export_to_markdown(result, "analysis.md")

# Filter specific information
critical_issues = get_critical_issues(result)
urgent_actions = get_urgent_actions(result)
```

### Use Cases

1. **Customer Support** - Analyze escalations and prioritize responses
2. **Incident Management** - Perform root cause analysis on outages
3. **HR Analytics** - Extract insights from employee feedback
4. **Product Management** - Analyze user feedback and feature requests
5. **Risk Assessment** - Identify and categorize business risks

### Customization

- Modify the prompt in `SimpleInsightAgent.__init__()` for domain-specific analysis
- Adjust Pydantic models to capture additional fields
- Change LLM temperature for more creative vs. deterministic analysis
- Add custom filtering and export functions